<a href="https://colab.research.google.com/github/arthur-b-martins/AI-Agents-Lamia/blob/main/pratica-2/groq_agent_pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install groq

In [ ]:
import os
import re
from typing import List, TypedDict
os.environ['GROQ_API_KEY'] = "gsk_uuEF8TCY"

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast cars",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

The thrill of the open road and the rush of adrenaline! While not everyone may share the enthusiasm for fast cars, there are certain benefits and importance to high-performance vehicles. Here are some points to consider:

1. **Innovation and Technology**: Fast cars push the boundaries of automotive technology, driving innovation in fields like aerodynamics, engine design, and materials engineering. These advancements often trickle down to mainstream vehicles, improving overall performance, safety, and efficiency.
2. **Safety**: Paradoxically, fast cars are often safer than their slower counterparts. They typically feature advanced safety features, such as high-performance brakes, improved crash structures, and sophisticated active safety systems.
3. **Performance and Handling**: Fast cars provide a unique driving experience, allowing drivers to explore the limits of their skills and enjoy the thrill of acceleration, cornering, and braking. This can improve driving skills and situationa

In [ ]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    completion = client.chat.completions.create(
        messages=self.messages,
        model="llama3-70b-8192"
    )
    return completion.choices[0].message.content



In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_all_from:
e.g. get_all_from:STOCK
returns the whole table of products in stock
there are two tables: STOCK, STORE

get_filtered_from:
e.g get_filtered_from:STORE,"quantity", lambda x:x < 10
returns the items of products in store with quantity less than 10.
there are three fields: "quantity", "name", "price"
you must always use get_filtered_from because it is more efficient.
be carefull with lambda expressions, you can not call Observation as
a list.

Example session:

Question: is there any product missing in store that we have in stock?
Thought: I need to check the store
Action: get_all_from:STORE
PAUSE

You will be called again with this:

Observation: [
    {"name": "milk", "quantity": 3, "price": 2.5},
    {"name": "pasta", "quantity": 13, "price": 1.5},
    {"name": "rice", "quantity": 0, "price": 12},
    {"name": "sugar", "quantity": 12, "price": 5},
    {"name": "flour", "quantity": 22, "price": 2.2},
    {"name": "coffee", "quantity": 0, "price": 25},
    {"name": "Oil", "quantity": 23, "price": 3},
]

Thought: I need to look for rice and coffee in stock
Action: get_filtered_from:STOCK,"name", lambda x: x == "rice" or x == "coffee"
PAUSE

You will be called again with this:

Observation: [
    {"name": "rice", "quantity": 12, "price": 12},
    {"name": "coffee", "quantity": 0, "price": 25},
]

If you have the answer, output it as the Answer.

Answer: hey! There isn't coffee and rice in your store anymore,
but there is rice in stock, you should bring it!!.

Now it's your turn:
""".strip()

In [ ]:
class Item(TypedDict):
  name: str
  quantity: int
  price: float

STOCK: List[Item] = [
    {"name": "milk", "quantity": 13, "price": 2.5},
    {"name": "pasta", "quantity": 213, "price": 1.5},
    {"name": "rice", "quantity": 132, "price": 12},
    {"name": "sugar", "quantity": 22, "price": 5},
    {"name": "flour", "quantity": 0, "price": 2.2},
    {"name": "coffee", "quantity": 20, "price": 25},
    {"name": "Oil", "quantity": 103, "price": 3},
]

STORE: List[Item] = [
    {"name": "milk", "quantity": 13, "price": 2.5},
    {"name": "pasta", "quantity": 3, "price": 1.5},
    {"name": "rice", "quantity": 17, "price": 12},
    {"name": "sugar", "quantity": 12, "price": 5},
    {"name": "flour", "quantity": 2, "price": 2.2},
    {"name": "coffee", "quantity": 10, "price": 25},
    {"name": "Oil", "quantity": 23, "price": 3},
]

def get_filtered_from(table: List[Item], field: str, lambdaFunction) -> str:
    return str([item for item in table if lambdaFunction(item[field])])

def get_all_from(table: List[Item]) -> str:
    return str(table)

In [ ]:
print(get_filtered_from(STORE, "quantity", lambda x: x < 10))

[{'name': 'milk', 'quantity': 3, 'price': 2.5}, {'name': 'rice', 'quantity': 0, 'price': 12}, {'name': 'coffee', 'quantity': 0, 'price': 25}]


In [ ]:
marketFriend = Agent(client=client, system=system_prompt)

In [ ]:
res = marketFriend("is there oil in stock")
print(res)

Thought: I need to check if oil is in stock
Action: get_filtered_from:STOCK,"name", lambda x: x == "oil"
PAUSE


In [ ]:
res = marketFriend("Observation: []")
print(res)

Thought: I didn't find oil in stock
Answer: Nope, there is no oil in stock.


In [ ]:
def agent_loop(max, system, query):
  agent = Agent(client, system)
  next_prompt = query
  tools = ['get_filtered_from', 'get_all_from']

  for i in range(max):
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result:
      action = re.findall(r"Action: ([a-z_]+):(.+)", result, re.IGNORECASE)
      chosen_tool = action[0][0]
      arg = action[0][1]

      if chosen_tool in tools:
        tool_response = eval(f"{chosen_tool}({arg})")
        next_prompt = f"Observation: {tool_response}"

      else:
        next_prompt = "Observation: tool not found"

      print(next_prompt)
      continue


    if "Answer" in result:
      break

    # clean next_prompt if the last was only a thought
    if "Thought" in result:
      next_prompt = ""

In [ ]:
agent_loop(20, system_prompt, "Do we have products less than 10 units in store? Which of them we can bring more from the stock?")

Thought: I need to check the store for products with less than 10 units.

Action: get_filtered_from:STORE,"quantity", lambda x: x < 10
PAUSE
Observation: [{'name': 'pasta', 'quantity': 3, 'price': 1.5}, {'name': 'flour', 'quantity': 2, 'price': 2.2}]
Thought: I need to check if we have more of these products in stock.

Action: get_filtered_from:STOCK,"name", lambda x: x in ["pasta", "flour"]
PAUSE
Observation: [{'name': 'pasta', 'quantity': 213, 'price': 1.5}, {'name': 'flour', 'quantity': 0, 'price': 2.2}]
Thought: It seems we can bring more pasta from stock.

Answer: Yes, we have products less than 10 units in store and we can bring pasta from stock.
